In [1]:
SIMULATION_DATE = "2025-07-15"

from config import r,STOCKS,FILEPATH,s3,EXCHANGE
import os
def download_file():
    if not os.path.exists(f'{FILEPATH}/{EXCHANGE}/{STOCKS[0]}/{SIMULATION_DATE}.csv'):
        try:
            s3.download_file(Bucket='kite', Key=f'{EXCHANGE}/{STOCKS[0]}/{SIMULATION_DATE}.csv', Filename=f'{FILEPATH}/{EXCHANGE}/{STOCKS[0]}/{SIMULATION_DATE}.csv')
            print(f"[MAIN] Downloaded {EXCHANGE}:{STOCKS[0]} for {SIMULATION_DATE}")
        except Exception as e:
            raise Exception("file not found,choose another date.")
download_file()

from simulator import get_all_tick_data
df = get_all_tick_data(SIMULATION_DATE)

from StockAnalyser import Cumulative_Support
instance = Cumulative_Support()
import pandas as pd
for row in df.rolling(1):
    instance.parse(row.iloc[0].to_dict())


[INFO] loaded data into memory for simulation


/Users/gurusai/programming/STONKS/BackTestingEngine/app/StockAnalyser.py:103: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.ltpDf = pd.concat([self.ltpDf, pd.DataFrame(new_record)], ignore_index=True)


In [ ]:
import pyqtgraph as pg
import pandas as pd
import numpy as np


app = pg.mkQApp()
temp = instance.combineddf[1].to_numpy()+4*instance.combineddf[0].to_numpy()
win_buy = pg.GraphicsLayoutWidget(title="Buy Heatmap & Trends")
win_buy.setBackground("white")
# ======== CREATE PLOTS =======
plot_buy = win_buy.addPlot(title="Buy Side")
plot_diff = win_buy.addPlot(title="DIFF",row=1, col=0)
win_buy.show()
plot_buy.setXLink(plot_diff)
# ======== HELPER FUNCTIONS =======
def addHeatMap(plot,data):
    # Create a colormap with transparency
    """
    colors = [
        (1, (0, 255, 0, 200)), # Green with light alpha
        (0, (255, 0, 0, 200))    # Red with light alpha
    ]
        
    cmap = pg.ColorMap(pos=np.array([c[0] for c in colors]), color=np.array([c[1] for c in colors], dtype=np.ubyte))
    img = pg.ImageItem(data)
    img.setOpacity(.5)
    lut = cmap.getLookupTable(nPts=256, alpha=True)  # Enable alpha
    img.setLookupTable(lut)
    plot.addItem(img)
    return img"""
    lut = np.array([
        #   R,   G,   B,   A
        [255,   0,   0, 170],   #  0: Red           → sell: top neg, bottom neg | buy: top neg, bottom neg      buy: red         sell: red
        [  0, 114, 178, 170],   #  1: Blue          → sell: top neg, bottom pos | buy: top neg, bottom neg      buy: brown       sell: red
        [230, 159,   0, 170],   #  2: Orange        → sell: top pos, bottom neg | buy: top neg, bottom neg      buy: green       sell: red
        [ 86, 180, 233, 170],   #  3: Sky Blue      → sell: top pos, bottom pos | buy: top neg, bottom neg      buy: neon green  sell: red
        [  0, 158, 115, 170],   #  4: Bluish Green  → sell: top neg, bottom neg | buy: top neg, bottom pos      buy: red         sell: brown
        [204, 121, 167, 170],   #  5: Purple        → sell: top neg, bottom pos | buy: top neg, bottom pos      buy: brown       sell: brown
        [240, 228,  66, 170],   #  6: Yellow        → sell: top pos, bottom neg | buy: top neg, bottom pos      buy: green       sell: brown
        [139,  69,  19, 170],   #  7: Brown         → sell: top pos, bottom pos | buy: top neg, bottom pos      buy: neon        sell: brown
        [255, 105, 180, 170],   #  8: Pink          → sell: top neg, bottom neg | buy: top pos, bottom neg      buy: red         sell: green
        [ 27,  94,  32, 170],   #  9: Dark Green    → sell: top neg, bottom pos | buy: top pos, bottom neg      buy: brown       sell: green
        [  0,   0, 128, 170],   # 10: Navy          → sell: top pos, bottom neg | buy: top pos, bottom neg      buy: green       sell: green
        [128,   0,   0, 170],   # 11: Maroon        → sell: top pos, bottom pos | buy: top pos, bottom neg      buy: neon green  sell: green
        [  0, 128, 128, 170],   # 12: Teal          → sell: top neg, bottom neg | buy: top pos, bottom pos      buy: red         sell: neon green
        [245, 245, 245, 170],   # 13: Off-White     → sell: top neg, bottom pos | buy: top pos, bottom pos      buy: brown       sell: neon green
        [119, 119, 119, 170],   # 14: Gray          → sell: top pos, bottom neg | buy: top pos, bottom pos      buy: green       sell: neon green
        [ 57, 255,  20, 170],   # 15: Neon Green    → sell: top pos, bottom pos | buy: top pos, bottom pos      buy: neon green  sell: neon green
    ], dtype=np.ubyte)
    # Create the ImageItem
    img = pg.ImageItem(data)

    # Assign the LUT
    img.setLookupTable(lut)

    # No need for setOpacity()—alpha comes from LUT
    plot.addItem(img)
    return img
def getXY(data: pd.Series):
    if data is None or data.empty or data.shape == 0:
        return [], []
    if isinstance(data, pd.DataFrame):
    # Take first column automatically
        data = data.iloc[:, 0]
    x = instance.ltpDf.index.to_numpy()
    
    # Convert to numeric if data is not already float or int
    if data.dtypes not in [np.float64, np.int64]:  
        y = pd.to_numeric(data, errors='coerce').to_numpy()
    else:
        y = data.to_numpy()

    # Ensure x and y have the same length
    min_len = min(len(x), len(y))
    x, y = x[:min_len], y[:min_len]  

    # Create mask and apply it safely
    mask = ~np.isnan(y)
    mask = mask[:min_len]  # Ensure mask has the same length as x and y

    return x[mask], y[mask]  # Filter out NaN values        
def addlinePlot(plot,linewidth,data:pd.Series,name,color=None):
    x,y = getXY(data)
    if color==None:
        return plot.plot(x,y,pen=pg.mkPen( width=linewidth,name=name))
    return plot.plot(x,y,pen=pg.mkPen(color=color, width=linewidth,name=name))

# ======= CREATE ELEMENTS =========
#BUY
instance.hmap_buy = addHeatMap(plot_buy, temp.T)
instance.line_upper_1_buy = addlinePlot(plot_buy, linewidth=4, data=instance.lowHighMaxes[0], name='Buy Uptrend', color='#097969')
instance.line_lower_1_buy = addlinePlot(plot_buy, linewidth=4, data=instance.HighlowMaxes[0], name='Buy Downtrend', color='#fbd604')
instance.line_ltp_buy = addlinePlot(plot_buy, linewidth=2, data=instance.ltpDf['ltp'], name='LTP', color=(0, 0, 255))
#DIFF
instance.vol_diff_50 = addlinePlot(plot_diff, linewidth=4, data=instance.voldiff_buy, name='VolDiff_buy', color='#fdd750')
instance.vol_diff_20 = addlinePlot(plot_diff, linewidth=4, data=instance.voldiff_sell, name='VolDiff_sell', color='#097969')

# ======== UPDATE VALUES ========
#heatmap
instance.hmap_buy.setImage(temp.T, autoLevels=False)
min_ltp = instance.ltpDf['ltp'].min()
n = len(instance.ltpDf)
m = len(instance.aggDf)
instance.hmap_buy.setRect(0, min_ltp, n, m)
#ltp
x,y_ltp = getXY(instance.ltpDf['ltp'])
instance.line_ltp_buy.setData(x,y_ltp)

#DIFF
x,y = getXY(instance.voldiff_buy)
instance.vol_diff_50.setData(x,y)
x,y = getXY(instance.voldiff_sell)
instance.vol_diff_20.setData(x,y)

app.exec()


2025-07-24 01:02:08.500 python[28734:36320076] +[IMKClient subclass]: chose IMKClient_Legacy
2025-07-24 01:02:08.500 python[28734:36320076] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


0